# Upload Data to MyCoast


(https://gist.github.com/pugwonk/ec192de6814486db678bdbb8fed0937b)

## 1. Start with Airtable

In [1]:
import pandas as pd
import os

survey_link = r"D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Survey_wPhotoLinks.csv"
photos_folder = r"D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Convert"

df = pd.read_csv(survey_link)
df = df.drop(['Photo','S123_Object_ID'], axis=1 )
df['Date'] = pd.to_datetime(df['Date'])
df.head()

c:\Users\cansu\.conda\envs\Data_Analysis\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\cansu\.conda\envs\Data_Analysis\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
c:\Users\cansu\.conda\envs\Data_Analysis\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


,unique_id,Date,Address,Neighborhood,Latitude,Longitude,Contact,Event,Source,Approx. Depth,What is affected by the flood waters?,What are additional causes of flooding?,Weather,Additional Comments,May we contact you in the future for any of the following reasons?
0,2,2018-09-10 09:50:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534008,-73.830255,B. Kissane,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
1,3,2018-09-10 22:35:00,First Street,Howard Beach/ Hamilton Beach,40.6541251,-73.830871,G. Ohringer,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
2,4,2018-09-10 23:35:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534843,-73.830255,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
3,5,2018-09-10 23:35:00,Rau Court,Howard Beach/ Hamilton Beach,40.6539708,-73.829887,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN
4,6,2018-09-25 09:45:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534572,-73.829909,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN


Combine tabular data with file names for photos. 

In [2]:
all_photos = os.listdir( photos_folder )

for i,r in df.iterrows():
    row_id = r['unique_id']
    
    rows_photos = [ i for i in all_photos if i.split('-')[0] == str(row_id) ]
    df.loc[i,'Photo_filenames'] = ", ".join(rows_photos)

df.head()

C:\Users\cansu\AppData\Local\Temp\ipykernel_5968\1379065450.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2-0.jpg, 2-1.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i,'Photo_filenames'] = ", ".join(rows_photos)


,unique_id,Date,Address,Neighborhood,Latitude,Longitude,Contact,Event,Source,Approx. Depth,What is affected by the flood waters?,What are additional causes of flooding?,Weather,Additional Comments,May we contact you in the future for any of the following reasons?,Photo_filenames
0,2,2018-09-10 09:50:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534008,-73.830255,B. Kissane,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,"2-0.jpg, 2-1.jpg"
1,3,2018-09-10 22:35:00,First Street,Howard Beach/ Hamilton Beach,40.6541251,-73.830871,G. Ohringer,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,3-0.jpg
2,4,2018-09-10 23:35:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534843,-73.830255,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,4-0.jpg
3,5,2018-09-10 23:35:00,Rau Court,Howard Beach/ Hamilton Beach,40.6539708,-73.829887,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,5-0.jpg
4,6,2018-09-25 09:45:00,Davenport Court,Howard Beach/ Hamilton Beach,40.6534572,-73.829909,C. O'Dea,September 2018 New Moon,Overland Flooding,NaN,NaN,NaN,NaN,NaN,NaN,6-0.jpg


In [3]:

df['What is affected by the flood waters?'] = df['What is affected by the flood waters?'].str.replace('_', ' ')
df['What is affected by the flood waters?'] = df['What is affected by the flood waters?'].str.replace( 'roads streets' , 'Roadway/street')

df['What is affected by the flood waters?'] = df['What is affected by the flood waters?'].str.replace(', ', ',')
df['What is affected by the flood waters?'] = df['What is affected by the flood waters?'].str.replace(',', ', ')

df['What is affected by the flood waters?'].unique()

array([nan, 'Roadway/street, Sidewalk, boardwalk', 'Roadway/ street',
       'Roadway/street, Sidewalk', 'Nothing', 'Sidewalk',
       'Roadway/street, Sidewalk, parking lot, Entrance to train station',
       'Roadway/street, sidewalks, parking lots, property',
       'Resurfacing of the roads; it was postponed until the next day due to water levels',
       'Roadway/street', 'Roadway/street, sidewalks', 'property',
       'Roadway/street, sidewalks, property', 'sidewalks',
       'Roadway/street, property', 'Roadway/street, parking lots',
       'parking lots', 'Roadway/street, sidewalks, other',
       'Roadway/street, sidewalks, parking lots, property, other',
       'Roadway/street, parking lots, property',
       'Roadway/street, sidewalks, parking lots',
       'Roadway/street, sidewalks, property, other'], dtype=object)

# Resize images  
Make sure **NOT** to work on the original images, create a new resize_folder.  
This code overrides the exsisting files in the folder. 

`max_size` defines the max size allowed on either dimension of the image.

In [93]:
from PIL import Image
max_size = 720

# There are reports without images attached to them.
for i,r in df[df['Photo_filenames']!='' ].iterrows():

    row_id = r['unique_id']
    row_photos = r['Photo_filenames'].split(', ')

    row_photos_fullpath = [ os.path.join(photos_folder, i) for i in row_photos ]

    for img_path in row_photos_fullpath: 
        if img_path != '': 

            image = Image.open( img_path )

            # Image dimensions 
            x = image.size[0]
            y = image.size[1]
            ratio = x/y
            print( img_path )

            if (x > max_size) | (y > max_size ):
                # Make sure the image is RGB. 
                image = image.convert('RGB')

                # limit longest side to 1000px
                if x > y:
                    new_x = max_size
                    new_y = int(max_size/ratio)
                else:
                    new_y = max_size
                    new_x = int(max_size*ratio)

                try:
                    image_resized = image.resize( (new_x, new_y) )
                    image_resized.save( img_path )
                except:
                    print( "Error resizing image: ", img_path )
                    image.save( img_path )
                    

D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\2-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\2-1.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\3-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\4-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\5-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\6-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\7-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\8-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\9-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\10-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\11-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Resized\12-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Ai

Clean up Source column. It is not written well.
**Overland flooding (Nearby waterbody)**  

Overland Flooding  
Overland Flooding Water Coming  

**Rainfall (Water pooling from direct rainfall)**  
Heavy Rains And No Known Source Of Drainage  
Heavy Rain And Storm Drains  
Rainfall Water Pooling  

**Storm drain (water pooling near a storm drain)**  
Storm Drainage Source  
Storm Drain/ Catch Basin  
Storm Drain Water  

**Broken pipes/water main break**  
Flooding From Pipes  

**Unsure**  
Not Sure  
Multiple Causes  

**Other**  


In [5]:
df['Source'] = df['Source'].str.replace('0','' )
df['Source'] = df['Source'].str.replace('_',' ' )
df['Source'] = df['Source'].str.replace('other','Other' )
df['Source'] = df['Source'].str.strip()
df['Source'] = df['Source'].fillna( '' )

df['Source'] = df['Source'].str.replace( 'Overland Flooding' , 'Overland flooding (Nearby waterbody)' )
df['Source'] = df['Source'].str.replace( 'overland flooding flooding from' , 'Overland flooding (Nearby waterbody)' )
df['Source'] = df['Source'].str.replace( 'overland flooding water coming' , 'Overland flooding (Nearby waterbody)' )

df['Source'] = df['Source'].str.replace( 'Heavy Rains and No known source of drainage' , 'Rainfall (Water pooling from direct rainfall)' )
df['Source'] = df['Source'].str.replace( 'Heavy Rain and Storm Drains' , 'Rainfall (Water pooling from direct rainfall)' )
df['Source'] = df['Source'].str.replace( 'rainfall water pooling from dir' , 'Rainfall (Water pooling from direct rainfall)' )

df['Source'] = df['Source'].str.replace( 'storm drain water coming from s' , 'Storm drain (Water pooling near a storm drain)' )
df['Source'] = df['Source'].str.replace( 'Storm Drainage Source' , 'Storm drain (Water pooling near a storm drain)' )
df['Source'] = df['Source'].str.replace( 'Storm Drain/ Catch Basin' , 'Storm drain (Water pooling near a storm drain)' )

df['Source'] = df['Source'].str.replace( 'Flooding from pipes' , 'Broken pipes/water main break' )

df['Source'] = df['Source'].str.replace( 'multiple causes if multiple cau' , 'Unsure' )
df['Source'] = df['Source'].str.replace( 'not sure' , 'Unsure' )

[i for i in df['Source'].unique()]

['Overland flooding (Nearby waterbody)',
 'Storm drain (water pooling near a storm drain)',
 'Rainfall (Water pooling from direct rainfall)',
 '',
 'Unsure',
 'Broken pipes/water main break',
 'Other']

In [6]:
df['Approx. Depth'] = df['Approx. Depth'].str.replace('in','') 
df['Approx. Depth'] = df['Approx. Depth'].str.replace('_ches','') 
df['Approx. Depth'] = df['Approx. Depth'].str.replace(' ches','') 
df['Approx. Depth'] = df['Approx. Depth'].str.replace('_','-') 

df['Approx. Depth'].unique()

array([nan, '6', '4', '8', '5', '2', '15', '12', '16', '3', '10', '8-12',
       '20', '0', '9', '18', '4.75', '14.5', '7', '6.5', '3.5', '1',
       '0.5', '2 ', '1-6', '6-12', '12-24', '<1', '24', '17', '11', '19'],
      dtype=object)

In [7]:
def convert_float(x):
    try:
        return float(x)
    except:
        return None

df['Latitude'] = df['Latitude'].apply( lambda x: convert_float(x) )
df['Longitude'] = df['Longitude'].apply( lambda x: convert_float(x) )
df['Longitude'].unique()

df = df[ (df['Latitude'].notnull()) & (df['Longitude'].notnull()) ]

In [17]:
df.tail()

,unique_id,Date,Address,Neighborhood,Latitude,Longitude,Contact,Event,Source,Approx. Depth,What is affected by the flood waters?,What are additional causes of flooding?,Weather,Additional Comments,May we contact you in the future for any of the following reasons?,Photo_filenames
511,513,2021-06-24 21:24:00,NaN,NaN,40.609452,-73.818586,Patrick Meyers,NaN,,5,Roadway/street,NaN,"sunny,light_wind",0,0,"513-0.jpg, 513-1.jpg"
512,514,2021-06-24 21:25:00,NaN,NaN,40.609046,-73.817455,Patrick Meyers,NaN,,0,Roadway/street,NaN,"sunny,light_wind",0,0,514-0.jpg
513,515,2021-06-24 21:28:00,NaN,NaN,40.609216,-73.817965,Patrick Meyers,NaN,,7,"Roadway/street, sidewalks",NaN,"sunny,light_wind",0,0,"515-0.jpg, 515-1.jpg, 515-2.jpg"
514,516,2021-06-24 21:32:00,NaN,NaN,40.658594,-73.832037,KG,NaN,,8,"Roadway/street, sidewalks",NaN,sunny,June supermoon,0,516-0.jpg
515,517,2021-06-24 21:50:00,NaN,NaN,40.654388,-73.831472,KG,NaN,,5,"Roadway/street, parking lots, property",NaN,light_wind,June Supermoon,0,"517-0.jpg, 517-1.jpg, 517-2.jpg"


In [16]:
import os
import uuid
import requests
import json
import time

def isnan(value):
  try:
      import math
      return math.isnan(float(value))
  except:
      return False

# ---- Test Users ----
# myCoastUserId = 13213
# myCoastUserHash = "$P$BEnD9bUs3T4TcaMykJ7D/TnAjVmekp1"
# form_id = 72

# ---- NY Anon ----
myCoastUserId = 9860
myCoastUserHash = "$P$BQQLTdlS2/AO.60WWx4A9hfVAlR06o1"
form_id = 92

# ---------------- Open and Upload Photos --------------
counter = 0
for i,r in df[ (df['Photo_filenames'].notnull()) & (df.index > 399 ) ].iterrows():
    counter = counter + 1
    print( f"Row: {counter}")

    row_id = r['unique_id']

    row_photos = r['Photo_filenames'].split(', ')
    row_photos_fullpath = [ os.path.join(photos_folder, i) for i in row_photos ]
    image_ids = [str(uuid.uuid4()) for _ in row_photos_fullpath ] # generate unique ids for each image

    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0", "Accept": "*/*" }

    # Define the URL
    for image_id, full_path in zip(image_ids , row_photos_fullpath):

        url = f"https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user={myCoastUserId}&hash={myCoastUserHash}&upload=image&id={image_id}&purpose=0"
        
        with open (full_path, 'rb') as file:
            file_content = file.read()
            response = requests.post( url, files = {'upfile': ('upfile', file_content)} , headers=headers )

            # Print the response
            if response.status_code == 200:
                print( full_path )
            else:
                print(f"Photo: Request failed with status code {response.status_code}")
                print( response.text )

    # ---------------------- REPORT ----------------------
    # Define the URL
    report_url = f"https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user={myCoastUserId}&hash={myCoastUserHash}&upload=report"
    
    what_is_affected = '' if isnan( r['What is affected by the flood waters?'] ) else r['What is affected by the flood waters?']
    what_is_affected = what_is_affected.split(', ')
    what_is_affected = [ i.strip() for i in what_is_affected ]
    what_is_affected = '' if what_is_affected[0] == '' else what_is_affected

    myReport = [
        {"name":"_gform-form-id", "value": form_id },
        {"name":"from_device", "value": "Can reports" },
        {"name":"photo_date", "value": r['Date'].strftime("%Y-%m-%d") },
        {"name":"photo_time", "value": r['Date'].strftime("%I:%M %p").lower() },
        {"name":"image1", "value": image_ids },
        {"name":"guessCause", "value": r['Source'] },
        {"name":"cause-other", "value": r['What are additional causes of flooding?'] },
        {"name":"flooded-other", "value": what_is_affected },
        {"name":"guessDepth", "value": r['Approx. Depth'] },
        {"name":"post_comment", "value": r['Additional Comments'] },
        {"name":"location_longitude", "value": float(r['Longitude']) },
        {"name":"location_latitude", "value": float(r['Latitude']) },
    ]

    for report_item in myReport:
        if (report_item['name'] == 'cause-other') or (report_item['name'] == 'flooded-other') :
            if isnan( report_item['value']):
                report_item['value'] = []
        else:
            if isnan( report_item['value']):
                report_item['value'] = ""

    # print all keys
    [ print(f"{i['name']} = {i['value']}") for i in myReport ]
    
    myReport = json.dumps(myReport)
    
    # Generate a unique report ID
    report_id = str(uuid.uuid4())

    # Send the POST request with JSON payload
    report_url = f"{report_url}&id={report_id}"

    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0", "Accept": "*/*", "Content-Type": "application/json" }
    response = requests.post( report_url , data=myReport , headers=headers )

    # Print the response
    if response.status_code == 200:
        print("Report OK")
    else:
        print(f"Report: Request failed with status code {response.status_code}")
    print( " ------------------- " )


    time.sleep(2)


Row: 1
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Convert\402-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Convert\402-1.jpg
_gform-form-id = 92
from_device = Can reports
photo_date = 2020-08-13
photo_time = 06:37 pm
image1 = ['270bc577-370c-4fdc-9997-1007ba2f9342', '7a3503cf-40f0-4819-b0b3-8112258e86de']
guessCause = Rainfall (Water pooling from direct rainfall)
cause-other = []
flooded-other = ['Roadway/street']
guessDepth = 2
post_comment = 0
location_longitude = -73.80187043
location_latitude = 40.86402748
Report OK
 ------------------- 
Row: 2
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Convert\403-0.jpg
D:\WORK\FLOODNET\WatchData\FloodWatch_Airtable\Airtable_Photos_Convert\403-1.jpg
_gform-form-id = 92
from_device = Can reports
photo_date = 2020-08-14
photo_time = 01:00 pm
image1 = ['144cb08d-e315-4591-a70f-3cd83031ed32', '2c03dced-6905-47c8-93b3-e6078a23f4d4']
guessCause = Unsure
cause-other = []
flooded-other = ['Ro

In [99]:
t = df.dropna(subset='Additional Comments', axis=0 )
t[t['Additional Comments'].str.contains('Water line is approximately 36') ]

,unique_id,Date,Address,Neighborhood,Latitude,Longitude,Contact,Event,Source,Approx. Depth,What is affected by the flood waters?,What are additional causes of flooding?,Weather,Additional Comments,May we contact you in the future for any of the following reasons?,Photo_filenames
92,94,2019-03-21 22:36:00,Intersection of East 6th road and Cross Bay bo...,Broad Channel,40.609544,-73.818958,P. Meyers,March 2019 Full Moon,,NaN,NaN,NaN,NaN,"Water line is approximately 36"" east of the st...",NaN,94-0.jpg


# Survey123

In [20]:
# Make sure photos are small and in JPEG format
from PIL import Image

photos123_folder = r"D:\WORK\FLOODNET\WatchData\FloodWatch_Survey123\Photos_Resized"
photos123 = os.listdir( photos123_folder )

max_size = 720
# There are reports without images attached to them.
for photo in photos123:

    row_photos_fullpath = os.path.join(photos123_folder, photo)
    image = Image.open( row_photos_fullpath )

    # Image dimensions 
    x = image.size[0]
    y = image.size[1]
    ratio = x/y

    if (x > max_size) | (y > max_size ):
        # Make sure the image is RGB. 
        image = image.convert('RGB')

        # limit longest side to 1000px
        if x > y:
            new_x = max_size
            new_y = int(max_size/ratio)
        else:
            new_y = max_size
            new_x = int(max_size*ratio)

        try:
            image_resized = image.resize( (new_x, new_y) )
            image_resized.save( row_photos_fullpath , format='JPEG' )
        except:
            print( "Error resizing image: ", photo )
            image.close()

In [90]:
import geopandas as gpd
import fiona

# Open Survey
path = r"C:\Users\cansu\Downloads\Survey123_Floodwatch_724entries(1)\Survey123_Floodwatch_724entries\S123_b9b1d621d16543378b6d3a6b3e02b424_FGDB\520636cb-9aef-45eb-9524-4bf049dd35fa.gdb"
layers = fiona.listlayers( path )

# Photos list. 
survey123_photos = pd.read_csv( r"D:\WORK\FLOODNET\WatchData\FloodWatch_Survey123\data_update\pho1.csv" ) 
gdf_survey = gpd.read_file( path , driver='FileGDB' , layer = 'survey')

gdf_survey.head(3)

,globalid,CreationDate,Creator,EditDate,Editor,which_of_the_following_are_floo,which_of_the_following_are_floo_other,please_describe_the_weather_con,please_describe_the_weather_con_other,additional_comments,contact_information,please_indicate_the_approximate,please_enter_the_current_time,please_enter_todays_date,what_caused_the_flooding_,what_caused_the_flooding__other,is_the_location_flooded_at_the_,contact_y_n,neighborhood_andor_borough_,geometry
0,{E4EC81F4-186D-41B0-8779-A0E938C814A3},2020-05-07 00:12:32+00:00,,2021-08-20 03:07:39+00:00,JBEditor,property,NaN,light_rain,NaN,NaN,NaN,NaN,12:00,2020-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.83798 40.65789)
1,{934FAB2F-3314-4ABC-8583-FF1A3B968439},2020-05-07 00:15:23+00:00,,2021-08-20 03:07:39+00:00,JBEditor,roads_streets,NaN,light_rain,NaN,tide water coming to the street from storm dra...,NaN,NaN,12:00,2020-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.83784 40.65775)
2,{0A4DB08D-AE85-441E-86B2-80842C85C4C9},2020-05-07 00:35:38+00:00,,2021-12-06 20:46:09+00:00,JBEditor,"roads_streets,sidewalks",NaN,"light_wind,light_rain",NaN,NaN,NaN,NaN,00:30,2020-05-07 00:00:00+00:00,Storm_drain_Water_bubbling_up_f,NaN,Yes,No,Hamilton Beach,POINT (-73.82944 40.65346)


In [94]:
pd.set_option('display.max_columns', None)

photos_list = gdf_survey[['globalid']].join( survey123_photos.set_index('REL_GLOBALID') , on='globalid', how='right' )
photos_list['GLOBALID'] = photos_list['GLOBALID'].apply( lambda x: x.replace("{","").replace("}","") )

photos_list.columns = ['globalid','ATTACHMENTID' ,'Photo_ID' , 'CONTENT_TYPE' ,'filename','data_size']

photos_list = photos_list.groupby('globalid').agg({'Photo_ID':list})

gdf_survey = gdf_survey.join( photos_list , on='globalid', how='left' )
gdf_survey = gdf_survey.drop(['Creator','CreationDate','EditDate','Editor'], axis=1 )

gdf_survey.head(3)

,globalid,which_of_the_following_are_floo,which_of_the_following_are_floo_other,please_describe_the_weather_con,please_describe_the_weather_con_other,additional_comments,contact_information,please_indicate_the_approximate,please_enter_the_current_time,please_enter_todays_date,what_caused_the_flooding_,what_caused_the_flooding__other,is_the_location_flooded_at_the_,contact_y_n,neighborhood_andor_borough_,geometry,Photo_ID
0,{E4EC81F4-186D-41B0-8779-A0E938C814A3},property,NaN,light_rain,NaN,NaN,NaN,NaN,12:00,2020-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.83798 40.65789),[1EAC122B-989F-468D-8253-366753782318]
1,{934FAB2F-3314-4ABC-8583-FF1A3B968439},roads_streets,NaN,light_rain,NaN,tide water coming to the street from storm dra...,NaN,NaN,12:00,2020-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.83784 40.65775),[813D68D7-3524-4597-8127-4D59B75243B3]
2,{0A4DB08D-AE85-441E-86B2-80842C85C4C9},"roads_streets,sidewalks",NaN,"light_wind,light_rain",NaN,NaN,NaN,NaN,00:30,2020-05-07 00:00:00+00:00,Storm_drain_Water_bubbling_up_f,NaN,Yes,No,Hamilton Beach,POINT (-73.82944 40.65346),[98938CA5-BA06-4A1C-8007-42657A5B17A0]


In [95]:
def isnan(value):
  try:
      import math
      return math.isnan(float(value))
  except:
      return False

gdf_survey['Latitude'] = gdf_survey['geometry'].y
gdf_survey['Longitude'] = gdf_survey['geometry'].x

gdf_survey['datetime'] = pd.to_datetime(gdf_survey['please_enter_todays_date'])
gdf_survey['date'] = gdf_survey['datetime'].dt.strftime( "%Y-%m-%d" )
gdf_survey['time'] = gdf_survey['datetime'].dt.strftime( "%I:%M %p" ).str.lower()

gdf_survey['Approx. Depth'] = gdf_survey['please_indicate_the_approximate'].apply( lambda x: str(int(x)) if not isnan(x) else '' )

gdf_survey.head(3)

,globalid,which_of_the_following_are_floo,which_of_the_following_are_floo_other,please_describe_the_weather_con,please_describe_the_weather_con_other,additional_comments,contact_information,please_indicate_the_approximate,please_enter_the_current_time,please_enter_todays_date,what_caused_the_flooding_,what_caused_the_flooding__other,is_the_location_flooded_at_the_,contact_y_n,neighborhood_andor_borough_,geometry,Photo_ID,Latitude,Longitude,datetime,date,time,Approx. Depth
0,{E4EC81F4-186D-41B0-8779-A0E938C814A3},property,NaN,light_rain,NaN,NaN,NaN,NaN,12:00,2020-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.83798 40.65789),[1EAC122B-989F-468D-8253-366753782318],40.657893,-73.837982,2020-05-07 00:00:00+00:00,2020-05-07,12:00 am,
1,{934FAB2F-3314-4ABC-8583-FF1A3B968439},roads_streets,NaN,light_rain,NaN,tide water coming to the street from storm dra...,NaN,NaN,12:00,2020-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.83784 40.65775),[813D68D7-3524-4597-8127-4D59B75243B3],40.657755,-73.837842,2020-05-07 00:00:00+00:00,2020-05-07,12:00 am,
2,{0A4DB08D-AE85-441E-86B2-80842C85C4C9},"roads_streets,sidewalks",NaN,"light_wind,light_rain",NaN,NaN,NaN,NaN,00:30,2020-05-07 00:00:00+00:00,Storm_drain_Water_bubbling_up_f,NaN,Yes,No,Hamilton Beach,POINT (-73.82944 40.65346),[98938CA5-BA06-4A1C-8007-42657A5B17A0],40.653456,-73.829444,2020-05-07 00:00:00+00:00,2020-05-07,12:00 am,


In [96]:
gdf_survey['What is affected by the flood waters?'] = gdf_survey['which_of_the_following_are_floo']

gdf_survey['What is affected by the flood waters?'] = gdf_survey['What is affected by the flood waters?'].str.replace('_', ' ')
gdf_survey['What is affected by the flood waters?'] = gdf_survey['What is affected by the flood waters?'].str.replace(', ', ',')
gdf_survey['What is affected by the flood waters?'] = gdf_survey['What is affected by the flood waters?'].str.replace(',', ', ')

gdf_survey['Additional Comments'] = gdf_survey['additional_comments']

gdf_survey['What is affected by the flood waters?'].unique()

array(['property', 'roads streets', 'roads streets, sidewalks',
       'roads streets, sidewalks, property', 'sidewalks',
       'roads streets, property', 'roads streets, parking lots',
       'roads streets, sidewalks, parking lots, property',
       'roads streets, sidewalks, other',
       'roads streets, sidewalks, parking lots, property, other',
       'parking lots', 'roads streets, parking lots, property',
       'roads streets, sidewalks, parking lots',
       'roads streets, sidewalks, property, other', nan,
       'roads streets, other', 'other', 'sidewalks, property'],
      dtype=object)

In [97]:
gdf_survey['What are additional causes of flooding?'] = gdf_survey['what_caused_the_flooding__other']

gdf_survey['What are additional causes of flooding?'] = gdf_survey['What are additional causes of flooding?'].fillna('')
gdf_survey['What are additional causes of flooding?'] = gdf_survey['What are additional causes of flooding?'].str.strip()
gdf_survey['What are additional causes of flooding?'] = gdf_survey['What are additional causes of flooding?'].str.replace( ',' , ' ' )

gdf_survey['What are additional causes of flooding?'].unique()

array(['', 'Manhole', 'Bike path', 'Lack of adequate drainage. ',
       'Fixing Bulkhead at the end of the block', 'Blocked Storm Drain',
       'Seems that water was coming up from ground even with 3 sump pumps',
       'Coming up from the floor', 'City Sewer backup',
       'Plus rain water compounded by blocked storm drains and low coastal area.',
       'Every time it rain this place over flows and all the homes get water back in there hone from this as well'],
      dtype=object)

**Overland flooding (Nearby waterbody)**  
Overland Flooding  
Overland Flooding Water Coming  

**Rainfall (Water pooling from direct rainfall)**  
Heavy Rains And No Known Source Of Drainage  
Heavy Rain And Storm Drains  
Rainfall Water Pooling  

**Storm drain (water pooling near a storm drain)**  
Storm Drainage Source  
Storm Drain/ Catch Basin  
Storm Drain Water  

**Broken pipes/water main break**  
Flooding From Pipes  

**Unsure**  
Not Sure  
Multiple Causes  

**Other**  


In [98]:
gdf_survey['Source'] = gdf_survey['what_caused_the_flooding_']

# Replace values to match MyCoast
gdf_survey['Source'] = gdf_survey['Source'].str.replace('Not_sure','Unsure' )
gdf_survey['Source'] = gdf_survey['Source'].str.replace('other','Other' ) 
gdf_survey['Source'] = gdf_survey['Source'].str.replace('Storm_drain_Water_bubbling_up_f','Storm drain (Water pooling near a storm drain)' ) 
gdf_survey['Source'] = gdf_survey['Source'].str.replace('Overland_flooding_Water_coming_','Overland flooding (Nearby waterbody)' ) 
gdf_survey['Source'] = gdf_survey['Source'].str.replace('Rainfall_Water_pooling_from_dir','Rainfall (Water pooling from direct rainfall)' ) 
gdf_survey['Source'] = gdf_survey['Source'].str.replace('Broken_pipes_water_main_break','Broken pipes/water main break' ) 

# Format Source
gdf_survey['Source'] = gdf_survey['Source'].apply( lambda x: x if not isnan(x) else '' )
gdf_survey['Source'] = gdf_survey['Source'].str.split(',')
gdf_survey['Source'] = gdf_survey['Source'].apply( lambda x: x[0] if len(x)==1 else x )

gdf_survey[ gdf_survey['Source'].notnull() ]['Source'].values

array(['', '', 'Storm drain (Water pooling near a storm drain)', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '',
       list(['Overland flooding (Nearby waterbody)', 'Storm drain (Water pooling near a storm drain)']),
       list(['Storm drain (Water pooling near a storm drain)', 'Other']),
       'Rainfall (Water pooling from direct rainfall)',
       'Storm drain (Water pooling near a storm drain)',
       list(['Overland flooding (Nearby waterbody)'

In [106]:
gdf_survey.head(3)

,globalid,which_of_the_following_are_floo,which_of_the_following_are_floo_other,please_describe_the_weather_con,please_describe_the_weather_con_other,additional_comments,contact_information,please_indicate_the_approximate,please_enter_the_current_time,please_enter_todays_date,what_caused_the_flooding_,what_caused_the_flooding__other,is_the_location_flooded_at_the_,contact_y_n,neighborhood_andor_borough_,geometry,Photo_ID,Latitude,Longitude,datetime,date,time,Approx. Depth,What is affected by the flood waters?,Additional Comments,What are additional causes of flooding?,Source
0,{E4EC81F4-186D-41B0-8779-A0E938C814A3},property,NaN,light_rain,NaN,NaN,NaN,NaN,12:00,2020-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.83798 40.65789),[1EAC122B-989F-468D-8253-366753782318],40.657893,-73.837982,2020-05-07 00:00:00+00:00,2020-05-07,12:00 am,,property,NaN,,
1,{934FAB2F-3314-4ABC-8583-FF1A3B968439},roads_streets,NaN,light_rain,NaN,tide water coming to the street from storm dra...,NaN,NaN,12:00,2020-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,POINT (-73.83784 40.65775),[813D68D7-3524-4597-8127-4D59B75243B3],40.657755,-73.837842,2020-05-07 00:00:00+00:00,2020-05-07,12:00 am,,roads streets,tide water coming to the street from storm dra...,,
2,{0A4DB08D-AE85-441E-86B2-80842C85C4C9},"roads_streets,sidewalks",NaN,"light_wind,light_rain",NaN,NaN,NaN,NaN,00:30,2020-05-07 00:00:00+00:00,Storm_drain_Water_bubbling_up_f,NaN,Yes,No,Hamilton Beach,POINT (-73.82944 40.65346),[98938CA5-BA06-4A1C-8007-42657A5B17A0],40.653456,-73.829444,2020-05-07 00:00:00+00:00,2020-05-07,12:00 am,,"roads streets, sidewalks",NaN,,Storm drain (Water pooling near a storm drain)


In [108]:
import os
import uuid
import requests
import json
import time


# ---- Test Users ----
myCoastUserId = 13213
myCoastUserHash = "$P$BEnD9bUs3T4TcaMykJ7D/TnAjVmekp1"
form_id = 72

# ---- NY Anon ----
# myCoastUserId = 9860
# myCoastUserHash = "$P$BQQLTdlS2/AO.60WWx4A9hfVAlR06o1"
# form_id = 92

# ---------------- Open and Upload Photos --------------
counter = 0
for i,r in gdf_survey.sample(3).iterrows():
    counter = counter + 1
    print( f"Row: {counter}")


    row_photos = r['Photo_ID']
    row_photos_fullpath = [ f"{os.path.join(photos123_folder, i)}.jpg" for i in row_photos ]

    image_ids = [str(uuid.uuid4()) for _ in row_photos_fullpath ] # generate unique ids for each image

    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0", "Accept": "*/*" }

    # Define the URL
    for image_id, full_path in zip(image_ids , row_photos_fullpath):

        url = f"https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user={myCoastUserId}&hash={myCoastUserHash}&upload=image&id={image_id}&purpose=0"
        
        with open (full_path, 'rb') as file:
            file_content = file.read()
            response = requests.post( url, files = {'upfile': ('upfile', file_content)} , headers=headers )

            # Print the response
            if response.status_code == 200:
                print( full_path )
            else:
                print(f"Photo: Request failed with status code {response.status_code}")
                print( response.text )

    # ---------------------- REPORT ----------------------
    # Define the URL
    report_url = f"https://mycoast.org/blueurchin-js/appthings.php?apiv=5&user={myCoastUserId}&hash={myCoastUserHash}&upload=report"
    
    what_is_affected = '' if isnan( r['What is affected by the flood waters?'] ) else r['What is affected by the flood waters?']
    what_is_affected = what_is_affected.split(', ')
    what_is_affected = [ i.strip() for i in what_is_affected ]
    what_is_affected = '' if what_is_affected[0] == '' else what_is_affected

    myReport = [
        {"name":"_gform-form-id", "value": form_id },
        {"name":"from_device", "value": "Can reports" },
        {"name":"photo_date", "value": r['date']},
        {"name":"photo_time", "value": r['time'] },
        {"name":"image1", "value": image_ids },
        {"name":"guessCause", "value": r['Source'] },
        {"name":"cause-other", "value": r['What are additional causes of flooding?'] },
        {"name":"flooded-other", "value": what_is_affected },
        {"name":"guessDepth", "value": r['Approx. Depth'] },
        {"name":"post_comment", "value": r['Additional Comments'] },
        {"name":"location_longitude", "value": float(r['Longitude']) },
        {"name":"location_latitude", "value": float(r['Latitude']) },
    ]

    for report_item in myReport:
        if (report_item['name'] == 'cause-other') or (report_item['name'] == 'flooded-other') :
            if isnan( report_item['value']):
                report_item['value'] = []
        else:
            if isnan( report_item['value']):
                report_item['value'] = ""

    # print all keys
    [ print(f"{i['name']} = {i['value']}") for i in myReport ]
    
    myReport = json.dumps(myReport)
    
    # Generate a unique report ID
    report_id = str(uuid.uuid4())

    # Send the POST request with JSON payload
    report_url = f"{report_url}&id={report_id}"

    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0", "Accept": "*/*", "Content-Type": "application/json" }
    response = requests.post( report_url , data=myReport , headers=headers )

    # Print the response
    if response.status_code == 200:
        print("Report OK")
    else:
        print(f"Report: Request failed with status code {response.status_code}")
    print( " ------------------- " )


    time.sleep(2)



Row: 1
D:\WORK\FLOODNET\WatchData\FloodWatch_Survey123\Photos_Resized\5B550409-2125-4A6B-8FDB-CACB2721EE33.jpg
_gform-form-id = 72
from_device = Can reports
photo_date = 2019-08-29
photo_time = 04:00 pm
image1 = ['af3540bf-c770-4c1d-8af9-5937d2999a49']
guessCause = Overland flooding (Nearby waterbody)
cause-other = 
flooded-other = ['roads streets', 'sidewalks', 'parking lots']
guessDepth = 
post_comment = 
location_longitude = -73.81981359999997
location_latitude = 40.597600200000045
Report OK
 ------------------- 
Row: 2
D:\WORK\FLOODNET\WatchData\FloodWatch_Survey123\Photos_Resized\7114411F-F086-49E0-8BFC-D13457BF0779.jpg
_gform-form-id = 72
from_device = Can reports
photo_date = 2018-12-21
photo_time = 04:00 pm
image1 = ['c68054f7-f561-46bd-ac66-3e26ad8d924e']
guessCause = Overland flooding (Nearby waterbody)
cause-other = 
flooded-other = ['roads streets', 'sidewalks']
guessDepth = 
post_comment = 
location_longitude = -73.77538359999994
location_latitude = 40.593009100000074
Repo